AICD provides membership lounges across the capitals in Australia. Members of AICD can visit these for meetings or for work. In the below presentation we'll analyze how members from different cities and industries visit AICD lounges. We'll also look for patterns based on members occupation and check if their companies office location and its distance to the local lounge has any influence on their behavior. 

<a id='index'></a>

# Inital analysis

In [1]:
import numpy as np 
import pandas as pd 
import geopy.distance
import matplotlib.pyplot as plt
# import plotly
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

df = pd.read_csv('members.csv')

# remove nulls and change negative avg_time to 'abs' int
df = df[~df['avg_dwell_time_in_min'].isnull()]
df['avg_dwell_time_in_min'] = abs(df['avg_dwell_time_in_min'])

# create a column with combined coordinates, and total_visits column: 
df['cords'] = tuple(zip(df['longitude'], df['latitude']))
df['total_visits'] = df['visits_sydney'] + df['visits_melbourne'] + df['visits_perth']

# remove total_visits nulls:
df = df[~(df['total_visits'] == 0) & ~(df['avg_dwell_time_in_min'] == 0)]

In [2]:
role_colors = ['#AB63FA', '#FFA15A', '#19D3F3','#FF6692']

fig = make_subplots(1, 2, specs=[[{'type':'domain'},{'type':'domain'}]],
                    subplot_titles=['Members job roles', 'Members HQ locations'])

fig.add_trace(go.Pie( values=df.groupby('role')['role'].count(), 
                     labels=['CEO','Chair','Exec Director','Non-Exec Director'] ,textinfo='label+percent', legendgroup = '1'), 1, 1)
fig.update_traces(marker=dict(colors=role_colors))

fig.add_trace(go.Pie(labels=df.groupby('headquarter_location')['role'].count().index, values=df.groupby('headquarter_location')['role'].count(),
                    textinfo='label+percent', legendgroup = '2'), 1, 2)
fig.update_layout(title_text='Plot 3: Member numbers by role and their headquarters location',legend_tracegroupgap = 80)
fig.show()

In [3]:
# AICD lounge locations:
coords_syd = (-33.86336382479448, 151.20641568508086)
coords_melb = (-37.81651586387018, 144.96262374491673)
coords_perth = (-31.955428859202144, 115.85814550988445)

# divide the dataset based on hq location:
hq_syd = df[df['headquarter_location'] == 'Sydney'].copy()
hq_melb = df[df['headquarter_location'] == 'Melbourne'].copy()
hq_perth = df[df['headquarter_location'] == 'Perth'].copy()

# calculate distances members hq to aicd lounge:
hq_syd['sydney_distance'] = hq_syd['cords'].apply(lambda x: geopy.distance.distance(x, coords_syd).km)
hq_melb['melb_distance'] = hq_melb['cords'].apply(lambda x: geopy.distance.distance(x, coords_melb).km)
hq_perth['perth_distance'] = hq_perth['cords'].apply(lambda x: geopy.distance.distance(x, coords_perth).km)

<a id='byhq'></a>
# Members visits by headquarters location
[back to top](#index)

In [4]:
# create a temp df for plotly:
plot5 = df.groupby('headquarter_location')[['visits_sydney','visits_melbourne','visits_perth']].sum().sort_values('visits_sydney',ascending=False)

labels_x = ['HQ in Sydney', 'HQ in Melbourne', 'HQ in Perth']
fig = go.Figure(data=[
    go.Bar(name='to Sydney', x=labels_x, y=plot5['visits_sydney']),
    go.Bar(name='to Melbourne', x=labels_x, y=plot5['visits_melbourne']),
    go.Bar(name='to Perth', x=labels_x, y=plot5['visits_perth']),
])

fig.update_layout(
    title='Plot 5: Visits by members headquarters location and destination',
    yaxis=dict(title='Number of visits'),
    xaxis=dict(title='HQ City'),
    barmode='group'
)
fig.show()

## Table 1: Members average visit numbers by their origin and destination 

In [5]:
df.groupby('headquarter_location')[['visits_sydney', 'visits_melbourne', 'visits_perth','total_visits']].mean().sort_values('visits_melbourne')

,visits_sydney,visits_melbourne,visits_perth,total_visits
headquarter_location,,,,
Sydney,15.302743,6.138186,6.137658,27.578586
Melbourne,11.669867,8.658789,11.765879,32.094535
Perth,11.446809,11.399420,13.320116,36.166344


## Table 2: Total visits by industry

In [6]:
cm = sns.light_palette("blue", as_cmap=True)
df.groupby('industry')[['visits_sydney','visits_melbourne','visits_perth','total_visits']].sum().sort_values('total_visits',ascending=False).style.background_gradient(cmap=cm, axis=0)

,visits_sydney,visits_melbourne,visits_perth,total_visits
industry,,,,
Financials,8687,5373,6311,20371
Health Care,8085,4942,6494,19521
Telecommunication,8133,5672,5565,19370
Media & Entertainment,8163,4895,6221,19279
Insurance,8048,5062,5333,18443
Energy,7727,4832,5750,18309
Software & Services,7808,4373,5667,17848


# Members visits by their roles and headquarters location
[back to top](#index)

In [7]:
# plot 11 visits by roles within local city:
fig = make_subplots(1, 3, specs=[[{'type':'domain'},{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['HQ in Sydney', 'HQ in Melbourne','HQ in Perth'])
fig.add_trace(go.Pie(labels=hq_syd.groupby('role')['visits_sydney'].sum().index, values=hq_syd.groupby('role')['visits_sydney'].sum(), scalegroup='one',
                     name="HQ in Sydney"), 1, 1)
fig.add_trace(go.Pie(labels=hq_melb.groupby('role')['visits_melbourne'].sum().index, values=hq_melb.groupby('role')['visits_melbourne'].sum(), scalegroup='one',
                     name="HQ in Melbourne"), 1, 2)
fig.add_trace(go.Pie(labels=hq_perth.groupby('role')['visits_perth'].sum().index, values=hq_perth.groupby('role')['visits_perth'].sum(), scalegroup='one',
                     name="HQ in Perth"), 1, 3)
fig.update_layout(title_text='Plot 11: Share of visits by role, within the same city')
fig.update_traces(marker=dict(colors=role_colors))
fig.show()

# plot 12 visits by roles outside local city:
fig = make_subplots(1, 3, specs=[[{'type':'domain'},{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['HQ in Sydney', 'HQ in Melbourne','HQ in Perth'])
fig.add_trace(go.Pie(labels=hq_syd.groupby('role')[['visits_melbourne','visits_perth']].sum().sum(axis=1).index, values=hq_syd.groupby('role')[['visits_melbourne','visits_perth']].sum().sum(axis=1), scalegroup='one',
                     name="HQ in Sydney"), 1, 1)
fig.add_trace(go.Pie(labels=hq_melb.groupby('role')[['visits_sydney','visits_perth']].sum().sum(axis=1).index, values=hq_melb.groupby('role')[['visits_sydney','visits_perth']].sum().sum(axis=1), scalegroup='one',
                     name="HQ in Melbourne"), 1, 2)
fig.add_trace(go.Pie(labels=hq_perth.groupby('role')[['visits_sydney','visits_melbourne']].sum().sum(axis=1).index, values=hq_perth.groupby('role')[['visits_sydney','visits_melbourne']].sum().sum(axis=1), scalegroup='one',
                     name="HQ in Perth"), 1, 3)

fig.update_traces(marker=dict(colors=role_colors))
fig.update_layout(title_text='Plot 12: Share of visits outside of HQ city by role')
fig.show()

### Table 6: Top 10 visitor types:

In [8]:
tot_v = df.groupby(['industry', 'headquarter_location','role'])[['visits_sydney','visits_melbourne','visits_perth','total_visits']].sum()
tot_v2 = df.groupby(['industry', 'headquarter_location','role'])['total_visits'].count().to_frame(name='members')
tot_v3 = df.groupby(['industry', 'headquarter_location','role'])['total_visits'].mean().to_frame(name='average visits')
tot_v4 = df.groupby(['industry','headquarter_location', 'role'])['avg_dwell_time_in_min'].mean().to_frame(name='average_time')
together = pd.concat([tot_v,tot_v2,tot_v3,tot_v4],axis=1)
together.sort_values('total_visits', ascending=False)[:10].style.background_gradient(cmap=cm)

,,,visits_sydney,visits_melbourne,visits_perth,total_visits,members,average visits,average_time
industry,headquarter_location,role,,,,,,,
Software & Services,Melbourne,Non-executive Director,1980,835,1921,4736,99,47.838384,145.333333
Media & Entertainment,Sydney,Chair,2468,1109,1138,4715,61,77.295082,155.098361
Telecommunication,Melbourne,Non-executive Director,1590,1418,1624,4632,108,42.888889,137.685185
Health Care,Sydney,Chair,2461,1037,1022,4520,58,77.931034,119.862069
Financials,Perth,Chair,1449,1420,1507,4376,50,87.520000,131.860000
Software & Services,Sydney,Chair,2231,1024,1021,4276,55,77.745455,139.745455
Telecommunication,Perth,Chair,1405,1430,1392,4227,48,88.062500,126.500000
Health Care,Melbourne,Non-executive Director,1487,1190,1474,4151,93,44.634409,136.849462
Financials,Melbourne,Non-executive Director,1545,1038,1546,4129,93,44.397849,128.784946


In [9]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>